# Data Modeling
Zuerst vorbereitete Daten einlesen

In [1]:
import pandas as pd;
import numpy as np;
from sklearn.model_selection import train_test_split
from sklearn import tree

cleaned_data = pd.read_csv('cleaned_data.csv')

### Robert 
--Predict the category of crime that occurred given a certain time and location

#### Location aufteilen in Dummy Varibalen 


In [2]:
# Überprüfen, ob "AREA.NAME" in den Spalten von "cleaned_data" vorhanden ist
if 'AREA.NAME' in cleaned_data.columns:
    # Spalte LOCATION wird zu mehrere 0,1 Variablen um diese im Suchbaum abzubilden 

    # One-Hot Encoding für das String-Feature "LOCATION"
    area_names = list(map(lambda x: 'AREA.NAME_' + x, set(cleaned_data['AREA.NAME'].values)))
    encoded_data = pd.get_dummies(cleaned_data, columns=['AREA.NAME'])
    cleaned_data = pd.concat([encoded_data], axis=1)
    # Anzeigen des transformierten DataFrames
    #print(area_names)
    cleaned_data['RD'] = cleaned_data['RD'].astype(str)




## DecisionTee

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import export_graphviz

#### DecisionTree 1/4
- features = ['RD']
- target = 'CRIME_VIOLENT'

In [6]:
predict_data = cleaned_data.copy()

# Schritt 2: Merkmale und Zielvariable definieren
features = ['RD', "LAT", "LONG"]
target = 'CRIME_VIOLENT'

# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: Modell erstellen und trainieren
model = DecisionTreeClassifier(max_depth = 8)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells:", accuracy)


Genauigkeit des Modells: 0.7413855505926675


#### DecisionTree 2/4
- features = ['LAT','LONG', 'RD', 'TIME.OCC_hour_cos']
- target = 'CRIME_CAT'

performt schlechter als ohne  'TIME.OCC_hour_cos' - siehe DecisionTee 3/4

In [26]:

predict_data = cleaned_data.copy()

# Schritt 2: Merkmale und Zielvariable definieren
features = ['LAT','LONG', 'RD', 'TIME.OCC_hour_cos']
target = 'CRIME_CAT'

# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: Modell erstellen und trainieren
model = DecisionTreeClassifier(max_depth = 80)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells:", accuracy)


Genauigkeit des Modells: 0.27553899431598516


#### DecisionTree 3/4
- features = [ 'RD',  'LAT', 'LONG']
- target = 'CRIME_CAT'


In [27]:
predict_data = cleaned_data.copy()

# Schritt 2: Merkmale und Zielvariable definieren
features = [ 'RD',  'LAT', 'LONG']
target = 'CRIME_CAT'


# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: Modell erstellen und trainieren
model = DecisionTreeClassifier(max_depth = 30)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells:", accuracy)

Genauigkeit des Modells: 0.30010117710929596


#### DecisionTree 4/4
- features = [ 'RD',  'LAT', 'LONG']
- target = 'Crm.Cd'


In [31]:
predict_data = cleaned_data.copy()

# Schritt 2: Merkmale und Zielvariable definieren
features = ['RD', 'LAT', 'LONG']
target = 'Crm.Cd'


# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: Modell erstellen und trainieren
model = DecisionTreeClassifier(max_depth = 40)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells:", accuracy)

Genauigkeit des Modells: 0.23439774942625174


## RandomForest


#### RandomForest 1/4
- features = ['RD', 'LAT', 'LONG']
- target = 'CRIME_VIOLENT'


In [34]:
predict_data = cleaned_data.copy()
from sklearn.ensemble import RandomForestClassifier
# Schritt 2: Merkmale und Zielvariable definieren
features = ['RD', 'LAT', 'LONG']
target = 'CRIME_VIOLENT'

# Remove all NaN values
predict_data = predict_data[predict_data['LAT'].notnull() & predict_data['LONG'].notnull()].copy()

# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: Modell erstellen und trainieren

model = RandomForestClassifier(max_depth = 20, n_estimators=70)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells: " , accuracy)

Genauigkeit des Modells:  0.742862078325889


#### RandomForest 2/4
- features = ['RD', 'LAT', 'LONG']
- target = 'CRIME_CAT'

Bisher bestes Ergebnis!

In [36]:
predict_data = cleaned_data.copy()
from sklearn.ensemble import RandomForestClassifier
# Schritt 2: Merkmale und Zielvariable definieren
features = ['RD', 'LAT', 'LONG']
target = 'CRIME_CAT'

# Remove all NaN values
predict_data = predict_data[predict_data['LAT'].notnull() & predict_data['LONG'].notnull()].copy()

# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: Modell erstellen und trainieren

model = RandomForestClassifier(max_depth = 24, n_estimators=64)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells: " , accuracy)
#Genauigkeit des Modells: deep: 24n_est: 64 0.3148376642071581

Genauigkeit des Modells:  0.3143729075668962


#### RandomForest 3/4
- features = ['RD', 'LAT', 'LONG']
- target = 'Crm.Cd'

In [12]:
predict_data = cleaned_data.copy()
from sklearn.ensemble import RandomForestClassifier
# Schritt 2: Merkmale und Zielvariable definieren
features = ['RD', 'LAT','LONG']
target = 'Crm.Cd'


# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

model = RandomForestClassifier(max_depth = 40, n_estimators= 64)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells:", accuracy)


Genauigkeit des Modells: 0.23849007559492963


#### RandomForest 4/4
- features = ['RD', 'LAT', 'LONG', 'TIME.OCC_hour_cos']
- target = 'Crm.Cd'

In [17]:
#Forrest

predict_data = cleaned_data.copy()
from sklearn.ensemble import RandomForestClassifier
# Schritt 2: Merkmale und Zielvariable definieren
features = ['RD', 'LAT', 'LONG', 'TIME.OCC_hour_cos']
target = 'Crm.Cd'

# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

model = RandomForestClassifier(max_depth = 80, n_estimators= 64)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells:", accuracy)

Genauigkeit des Modells: 0.1905913514136005


####  sequentielles neuronales Netzwerk verwendet.
- Dense-Schicht
- target = 'CRIME_CAT'
- features = 'AREA', 'RD', 'LAT', 'LONG',  'TIME.OCC_hour_cos', 'DATE.OCC_day_cos', 'DATE.OCC_month_cos' AND ALL FEATURES
- viel Ausprobiert - wenig gutes. 

In [30]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Schritt 1: Daten laden und vorverarbeiten
predict_data = cleaned_data.copy()
target = 'Crm.Cd'

features = ['RD', 'LONG', 'LAT', 'TIME.OCC_hour_cos']
# Remove all NaN values
predict_data = predict_data[predict_data['LAT'].notnull() & predict_data['LONG'].notnull()].copy()

# Schritt 2: Daten in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 5: Neuronales Netzwerk erstellen

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(16, activation='tanh', input_shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(16, activation='tanh'))

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train_encoded, epochs=2, batch_size=20, validation_split=0.2)

# Schritt 7: Vorhersagen treffen
y_pred = np.argmax(model.predict(X_test), axis=-1)

# Schritt 8: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells: " + str(accuracy))



ModuleNotFoundError: No module named 'tensorflow'

#### Naive Bayes-Klassifikator (GaussianNB)
- Target: CRIME_CAT
- Features: ['UNIX.TIMESTAMP','AREA.NAME','LAT','LONG']
- viel Ausprobiert - wenig gutes

In [23]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.naive_bayes import GaussianNB


predict_data = cleaned_data.copy()
# Build a Gaussian Classifier
nb_class = GaussianNB()


features = ['UNIX.TIMESTAMP','LAT','LONG']
target = 'CRIME_CAT'
selection = features + [target]

predict_data = cleaned_data[selection].copy()
# Filtern von null (muss nur für eines gemacht werden, da Koordinaten nie alleine vorkommen)
predict_data = predict_data[predict_data['LONG'].notnull()]


X_train, X_test, y_train, y_test = train_test_split(predict_data.drop(target, axis=1), predict_data[target], test_size=0.2, random_state=42)
nb_class.fit(X_train,y_train)

print("Model Accuracy:")
nb_class.score(X_test,y_test)

#probiert Priors anzupassen. 
#priors = [0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0565] - 8,9%
# nicht gut

#Auch weitere Anpassungen brachten keine Verbesserung:
#print("Accuracy (Standardized):", accuracy_standardized)
#print("Accuracy (Normalized):", accuracy_normalized)

Model Accuracy:


0.17517212447252178

### Kagetorischer Naive Bayers (CategoricalNB)
- features = ['TIME.OCC_hour','DATE.OCC_weekday','DATE.OCC_day','DATE.OCC_month','DATE.OCC_year','RD','LOCATION.street']
- target = 'CRIME_CAT'
- sehr viel besser

In [22]:
from sklearn.calibration import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.naive_bayes import CategoricalNB

# Build a Gaussian Classifier
nb_class = CategoricalNB()

features = ['TIME.OCC_hour','DATE.OCC_weekday','DATE.OCC_day','DATE.OCC_month','DATE.OCC_year','RD','LOCATION.street']
target = 'CRIME_CAT'
selection = features + [target]
predict_data_encoded = cleaned_data[selection].copy()
predict_data_encoded['LOCATION.street']= LabelEncoder().fit_transform(predict_data_encoded['LOCATION.street'])

X_train, X_test, y_train, y_test = train_test_split(predict_data_encoded.drop(target, axis=1), predict_data_encoded[target], test_size=0.2, random_state=42)
nb_class.fit(X_train,y_train)

print(f'Model Accuracy: {nb_class.score(X_test,y_test)}')

Model Accuracy: 0.2877419407908266


#### LogisticRegression

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

predict_data = cleaned_data.copy()

# Schritt 2: Merkmale und Zielvariable definieren
target = 'CRIME_CAT'
#column_to_exclude = ['Date.Rptd', 'DR.NO', 'DATE.OCC', 'TIME.OCC', 'AREA', 'AREA.NAME', 'RD', 'Crm.Cd', 'CrmCd.Desc', 'Status', 'Status.Desc', 'LOCATION', 'Cross.Street', 'LAT', 'LONG', 'UNIX.TIMESTAMP', 'TIME.OCC_hour', 'DATE.OCC_day', 'DATE.OCC_weekday', 'DATE.OCC_month', 'DATE.OCC_year', 'CRIME_VIOLENT', 'CRIME_CAT', 'Cos_Uhrzeit', 'TIME.OCC_hour_cos', 'DATE.OCC_day_cos', 'DATE.OCC_month_cos']
column_to_exclude = ['Date.Rptd', 'DR.NO', 'DATE.OCC', 'TIME.OCC', 'Crm.Cd', 'CrmCd.Desc', 'Status', 'Status.Desc', 'LOCATION.street', 
                     'Cross.Street.street', 'LOCATION.house_number', 'Cross.Street.house_number','UNIX.TIMESTAMP', 'TIME.OCC_hour', 
                     'DATE.OCC_day', 'DATE.OCC_weekday', 'DATE.OCC_month', 'DATE.OCC_year', 'CRIME_VIOLENT', 'CRIME_CAT']
#Enthalten: 'AREA', 'RD', 'LAT', 'LONG',  'TIME.OCC_hour_cos', 'DATE.OCC_day_cos', 'DATE.OCC_month_cos'
features = predict_data.drop([target] + column_to_exclude, axis=1).columns

# Remove all NaN values
predict_data = predict_data[predict_data['LAT'].notnull() & predict_data['LONG'].notnull()].copy()

# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: Logistische Regression initialisieren und trainieren
model = LogisticRegression(solver='liblinear', max_iter=1000)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren (optional)
accuracy = model.score(X_test, y_test)
print("Genauigkeitswert des Modells: " + str(accuracy))


#Genauigkeitswert des Modells: 0.41406762757950544

Genauigkeitswert des Modells: 0.1883539389153485


#### k-NN 
- target = 'Crm.Cd'

- features = ['RD', 'LONG', 'LAT', 'TIME.OCC_hour_cos']

In [27]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

predict_data = cleaned_data.copy()

target = 'CRIME_CAT'

features = ['RD', 'LONG', 'LAT']

# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

predict_data = predict_data[predict_data['LAT'].notnull() & predict_data['LONG'].notnull()].copy()
################

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV

# Schritt 2: Daten normalisieren
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Schritt 3: Feature Selection
selector = SelectKBest(k=3)  # Wähle die 10 besten Merkmale
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

# Schritt 4: Parameter Grid für die Grid Search festlegen
param_grid = {
    'n_neighbors': [15],
    'weights': ['distance'],
    'metric': ['manhattan']
}


# Schritt 5: Grid Search durchführen
model = KNeighborsClassifier()
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train_selected, y_train)

# Schritt 6: Beste Parameter und Genauigkeit ausgeben
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_
print("Parameter: ", best_params)
print("Genauigkeit: ", best_accuracy)

# Schritt 7: Vorhersagen treffen und Genauigkeit auf Testdaten berechnen
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit auf Testdaten: ", accuracy)


Parameter:  {'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'distance'}
Genauigkeit:  0.2864261100322282
Genauigkeit auf Testdaten:  0.29081427008530136
